In [16]:
import json
import cv2
import os

In [17]:
def get_images():
    images=os.listdir('./main_data/main_images')
    temp={}
    for i in images:
        temp[i]=cv2.imread(f'./main_data/main_images/{i}').copy()
    return temp

In [18]:
data=open('./main_data/main_json.json').read()
data=json.loads(data)
type(data)

dict

In [19]:
filter=['filename','x','y','width','height','0','1']

In [20]:
def get_depth(d):

    o = []
    def v_check(key,val):

        if key in filter or val in filter :
            o.append(str(f'{key}:{val}'))

    def check(d):
        if isinstance(d, dict):

            for k,v in d.items():

                if isinstance(v, (dict, list)):
                    check(v)

                else:
                    if v:
                       v_check(k,v)
                   

        elif isinstance(d, (list)):
            for v in d:

                if isinstance(v, (dict, list)):

                    check(v)

                else:
                    pass
                    # v_check(v)

    check(d)
    return o

data=get_depth(data)
# data

# YOLO PARSER

In [21]:
def normalize_for_yolo(xmin,ymin,w,h,w_img,h_img):
    xcenter = (xmin + w/2) / w_img
    ycenter = (ymin + h/2) / h_img
    w = w / w_img
    h = h / h_img
    return xcenter,ycenter,w,h
if not os.path.exists('./main_data/yolo_annotations'):
    os.mkdir('./main_data/yolo_annotations')
images=get_images()
image=''
file=''
points=[]
for i in data:
    # print(i)
    if i.split(':')[0] == 'filename':
        image=images[i.split(':')[1]]
        file=open(f'./main_data/yolo_annotations/{i.split(":")[1].split(".")[0]}.txt','w')
    else:
        if i.split(':')[0]=='0' or i.split(':')[0]=='1':
            file.write(f"{i.split(':')[0]} ")
            x,y,w,h=[int(x) for x in points]
            # image=cv2.rectangle(image,(x,y),(x+w,y+h),[0,255,0],2)
            # image=cv2.putText(image,i.split(':')[0],(x,y),cv2.FONT_HERSHEY_SIMPLEX,2,[0,255,0],2)
            # cv2.imshow('',image)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()
            x,y,w,h=normalize_for_yolo(x,y,w,h,image.shape[1],image.shape[0])
            file.write(f"{x} {y} {w} {h}\n")
            points=[]
        else:
            points.append(i.split(":")[1])
file.close()
    

# Unet

In [22]:
def create_mask_image(img):
    im=img.copy()
    im[:,:]=0
    return im
    

In [23]:
if not os.path.exists('./main_data/unet_annotations'):
    os.mkdir('./main_data/unet_annotations')
    if not os.path.exists('./main_data/unet_annotations/labels'):
        os.mkdir('./main_data/unet_annotations/labels')
    if not os.path.exists('./main_data/unet_annotations/train'):
        os.mkdir('./main_data/unet_annotations/train')

def mask_for_unet(filename,image,mask_image,points,color):
    x,y,w,h=[int(x) for x in points]
    cv2.rectangle(mask_image,(x,y),(x+w,y+h),color,-1)
    mask_image=cv2.resize(mask_image,(512,512),cv2.INTER_AREA)
    image=cv2.resize(image,(512,512),cv2.INTER_AREA)
    cv2.imwrite(f'./main_data/unet_annotations/labels/{filename}.jpg',mask_image)
    cv2.imwrite(f'./main_data/unet_annotations/train/{filename}.jpg',image)
    # cv2.imshow('',mask_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()


In [24]:
images=get_images()
image=''
mask_image=''
points=[]
for i in data:
    # print(i)
    if i.split(':')[0] == 'filename':
        image=images[i.split(':')[1]]
        filename=i.split(':')[1].split('.')[0]
        mask_image=create_mask_image(image)
    else:
        if i.split(":")[0] in ['x','y','width','height']:
            points.append(i.split(":")[1])
        else:
            c=255-int(i.split(":")[0])
            mask_for_unet(filename,image,mask_image,points,[c,c,c])
            points=[]